In [ ]:
%%bash
apt-get install openjdk-8-jdk-headless -qq > /dev/null

wget -q https://downloads.apache.org/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
tar xf spark-2.4.7-bin-hadoop2.7.tgz

pip install findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sparkContext=spark.sparkContext

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv("FIFA.csv",sep=",")
df.head(5)

,ID,lang,Date,Source,len,Orig_Tweet,Tweet,Likes,RTs,Hashtags,UserMentionNames,UserMentionID,Name,Place,Followers,Friends
0,1013597060640145408,en,2018-07-02 01:35:45,Twitter for Android,140,RT @Squawka: Only two goalkeepers have saved t...,Only two goalkeepers have saved three penaltie...,0,477,"WorldCup,POR,ENG",Squawka Football,Squawka,Cayleb,Accra,861,828
1,1013597056219295744,en,2018-07-02 01:35:44,Twitter for Android,139,RT @FCBarcelona: ?? @ivanrakitic scores the wi...,scores the winning penalty to send into the qu...,0,1031,WorldCup,"FC Barcelona,Ivan Rakitic,HNS | CFF","FCBarcelona,ivanrakitic,HNS_CFF",Febri Aditya,Bogor,667,686
2,1013597047482544130,en,2018-07-02 01:35:42,Twitter for Android,107,RT @javierfernandez: Tonight we have big game....,Tonight we have big game,0,488,worldcup,"Javier Fernandez,Evgeni Plushenko","javierfernandez,EvgeniPlushenko",??,NaN,65,67
3,1013597044198391808,en,2018-07-02 01:35:41,Twitter Web Client,142,We get stronger\r\nTurn the music up now\r\nWe...,We get stronger Turn the music up now We got t...,0,0,"PowerByEXO,WorldCup,FIFAStadiumDJ,XiuminLeague","EXO,FIFA World Cup ?","weareoneEXO,FIFAWorldCup",Frida Carrillo,"Zapopan, Jalisco",17,89
4,1013597039999926272,en,2018-07-02 01:35:40,Twitter for Android,140,RT @Squawka: Only two goalkeepers have saved t...,Only two goalkeepers have saved three penaltie...,0,477,"WorldCup,POR,ENG",Squawka Football,Squawka,tar,NaN,137,216


In [ ]:
from pyspark.sql.types import *

mySchema = StructType([  StructField("ID", StringType(), True)\
                        ,StructField("lang", StringType(), True)\
                        ,StructField("Date", StringType(), True)\
                        ,StructField("Source", StringType(), True)\
                        ,StructField("len", IntegerType(), True)\
                        ,StructField("Orig_Tweet", StringType(), True)\
                        ,StructField("Tweet", StringType(), True)\
                        ,StructField("Likes", IntegerType(), True)\
                        ,StructField("RTs", IntegerType(), True)\
                        ,StructField("Hashtags", StringType(), True)\
                        ,StructField("UserMentionNames", StringType(), True)\
                        ,StructField("UserMentionID", StringType(), True)\
                        ,StructField("Name", StringType(), True)\
                        ,StructField("Place", StringType(), True)\
                        ,StructField("Followers", IntegerType(), True)\
                        ,StructField("Friends", IntegerType(), True)])


sparkDF = spark.createDataFrame(df,schema=mySchema)
sparkDF.printSchema()
sparkDF.show()



root
 |-- ID: string (nullable = true)
 |-- lang: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- len: integer (nullable = true)
 |-- Orig_Tweet: string (nullable = true)
 |-- Tweet: string (nullable = true)
 |-- Likes: integer (nullable = true)
 |-- RTs: integer (nullable = true)
 |-- Hashtags: string (nullable = true)
 |-- UserMentionNames: string (nullable = true)
 |-- UserMentionID: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Place: string (nullable = true)
 |-- Followers: integer (nullable = true)
 |-- Friends: integer (nullable = true)

+-------------------+----+-------------------+-------------------+---+--------------------+--------------------+-----+----+--------------------+--------------------+--------------------+--------------------+--------------------+---------+-------+
|                 ID|lang|               Date|             Source|len|          Orig_Tweet|               Tweet|Likes| RTs|   

In [ ]:
import pyspark.sql.functions as f

hashtags = sparkDF.select('Hashtags').dropna()
hashtags = hashtags.select(f.split(hashtags.Hashtags, ",").alias('Hashtags'))
hashtags.groupBy('Hashtags').count().show(5)

+--------------------+-----+
|            Hashtags|count|
+--------------------+-----+
|  [CRODEN, WorldCup]| 2550|
|[FIFAStadiumDJ, W...|   30|
|     [WorldCup, RUS]|   35|
|[WorldCup, HalaMa...|  395|
|[promo, melbourne...|    1|
+--------------------+-----+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import explode
hashtags = hashtags.withColumn("Hashtags", explode(hashtags.Hashtags))
hashtags.show(5)

+--------+
|Hashtags|
+--------+
|WorldCup|
|     POR|
|     ENG|
|WorldCup|
|worldcup|
+--------+
only showing top 5 rows



In [ ]:
import pyspark.sql.functions as F
from pyspark.sql.functions import isnan

filteredHashtags = hashtags.filter(F.lower(hashtags.Hashtags) !="worldcup").dropna()
filteredHashtags = filteredHashtags.filter(~isnan(filteredHashtags.Hashtags))


10 наиболее упоминающихся хештегов


In [ ]:
filteredHashtags.groupBy('Hashtags').count().orderBy("count", ascending=False).show(10)

+-------------+-----+
|     Hashtags|count|
+-------------+-----+
|          FRA|66128|
|          CRO|32274|
|WorldCupFinal|28446|
|          ENG|25165|
|       FRAARG|21413|
|       FRABEL|18862|
|       FRACRO|17982|
|          BEL|15728|
|          ARG|15241|
|FIFAStadiumDJ|14200|
+-------------+-----+
only showing top 10 rows



Получить десять столиц государств, из которых наиболее часто посылались твиты. Список столиц нужно хранить в отдельном файле. 

In [ ]:
places = sparkDF.select('Place')
places = places.filter(~isnan(places.Place))
places = places.select(f.split(places.Place, ",").alias('Place'))
places.show(5)

+-------------------+
|              Place|
+-------------------+
|            [Accra]|
|            [Bogor]|
|[Zapopan,  Jalisco]|
|   [Mount Olympus ]|
|      [Lima,  Peru]|
+-------------------+
only showing top 5 rows



In [ ]:
places = places.withColumn("Place", explode(places.Place))
places.show(5)

+--------------+
|         Place|
+--------------+
|         Accra|
|         Bogor|
|       Zapopan|
|       Jalisco|
|Mount Olympus |
+--------------+
only showing top 5 rows



In [ ]:
capitals = ['London','Moscow','Berlin','Madrid','Copenhagen','Lisbon','Bogota','Paris',\
            'Dakar','Tokyo','Stockholm','Buenos Aires','Tunis','Warsaw','Seoul','Seoul',\
            'Belgrade','Zagreb','Brussels','Accra']
places1 = places.filter(F.col('Place').isin(capitals))
places1.groupBy('Place').count().orderBy("count", ascending=False).show(20)


+------------+-----+
|       Place|count|
+------------+-----+
|      London| 8334|
|       Accra| 1787|
|       Paris| 1432|
|      Madrid|  439|
|       Seoul|  365|
|Buenos Aires|  354|
|      Moscow|  347|
|      Berlin|  257|
|       Tokyo|  240|
|      Zagreb|  229|
|    Brussels|  203|
|   Stockholm|  168|
|  Copenhagen|   74|
|      Bogota|   71|
|    Belgrade|   63|
|      Lisbon|   46|
|      Warsaw|   31|
|       Dakar|   28|
|       Tunis|   10|
+------------+-----+



Определить наиболее часто употребляющийся хештег для каждой столицы из топ-10

> Блок с отступами






In [ ]:
topCapitals = ["London","Paris","Madrid","Seoul","Buenos Aires","Moscow","Tokyo","Zagreb","Accra","Berlin"]
capitalsHashtag = sparkDF.select(F.split(sparkDF.Hashtags, ",").alias('Hashtags'),"Place")
capitalsHashtag = capitalsHashtag.select(explode(capitalsHashtag.Hashtags).alias('Hashtags'),"Place")
capitalsHashtag.show(10)

capitalsHashtag = capitalsHashtag.select("Hashtags",F.split(sparkDF.Place, ",").alias('Place'),)
capitalsHashtag = capitalsHashtag.select("Hashtags",explode(capitalsHashtag.Place).alias('Place'))
capitalsHashtag.show(10)

capitalsHashtag = capitalsHashtag.filter(F.col('Place').isin(topCapitals))

capitalsHashtag = capitalsHashtag.filter(F.lower(capitalsHashtag.Hashtags) !="worldcup").dropna()
capitalsHashtag = capitalsHashtag.filter(~((isnan(capitalsHashtag.Hashtags) | isnan(capitalsHashtag.Place))))

capitalsHashtag.show(10)



+-------------+----------------+
|     Hashtags|           Place|
+-------------+----------------+
|     WorldCup|           Accra|
|          POR|           Accra|
|          ENG|           Accra|
|     WorldCup|           Bogor|
|     worldcup|             NaN|
|   PowerByEXO|Zapopan, Jalisco|
|     WorldCup|Zapopan, Jalisco|
|FIFAStadiumDJ|Zapopan, Jalisco|
| XiuminLeague|Zapopan, Jalisco|
|     WorldCup|             NaN|
+-------------+----------------+
only showing top 10 rows

+-------------+--------+
|     Hashtags|   Place|
+-------------+--------+
|     WorldCup|   Accra|
|          POR|   Accra|
|          ENG|   Accra|
|     WorldCup|   Bogor|
|     worldcup|     NaN|
|   PowerByEXO| Zapopan|
|   PowerByEXO| Jalisco|
|     WorldCup| Zapopan|
|     WorldCup| Jalisco|
|FIFAStadiumDJ| Zapopan|
+-------------+--------+
only showing top 10 rows

+---------+------------+
| Hashtags|       Place|
+---------+------------+
|      POR|       Accra|
|      ENG|       Accra|
|Cristiano|

In [ ]:
capitalHashtag1 = capitalsHashtag.groupBy('Place','Hashtags').count().orderBy("count", ascending=False)

In [ ]:
capitalHashtag1.show()

+------+-------------+-----+
| Place|     Hashtags|count|
+------+-------------+-----+
|London|          ENG|  972|
|London|          FRA|  824|
|London|   ThreeLions|  486|
|London|       ENGCRO|  429|
| Paris|          FRA|  400|
|London|          CRO|  394|
|London|WorldCupFinal|  333|
|London|       FRABEL|  330|
|London|       CRODEN|  265|
| Accra|          FRA|  244|
|London|          BEL|  237|
|London|       FRAARG|  235|
|London|       FRACRO|  198|
|London|      England|  185|
| Paris|       FRAARG|  168|
| Paris|WorldCupFinal|  167|
|London|       CROENG|  167|
| Paris|       FRABEL|  153|
|London|          ARG|  143|
|London|ItsComingHome|  140|
+------+-------------+-----+
only showing top 20 rows



In [ ]:
capHashtag = capitalHashtag1.join(capitalHashtag1.groupBy('Place').agg(F.max('count').alias('count')),on='count',how='leftsemi')

capHashtag.show()


+-----+------------+-------------+
|count|       Place|     Hashtags|
+-----+------------+-------------+
|   26|      London|       ESPRUS|
|   26|      Berlin|          FRA|
|   26|      London|          BRA|
|   26|      London|         thfc|
|   26|      London|          cro|
|   50|       Accra|       CRODEN|
|   50|      Moscow|          FRA|
|   50|       Tokyo|          FRA|
|   88|Buenos Aires|FIFAStadiumDJ|
|  104|      Zagreb|          CRO|
|   48|      London|   threelions|
|   48|      Madrid|          FRA|
|  100|       Seoul|FIFAStadiumDJ|
|  400|       Paris|          FRA|
|  244|       Accra|          FRA|
|  972|      London|          ENG|
+-----+------------+-------------+



In [ ]:
from pyspark.sql.window import Window
from pyspark.sql import Row
from pyspark.sql.functions import *


window = Window.partitionBy("Place").orderBy('tiebreak')
 
CapHash = capitalHashtag1.withColumn('tiebreak', monotonically_increasing_id())\
 .withColumn('rank', rank().over(window))\
 .filter(col('rank') == 1).drop('rank','tiebreak')

CapHash.show()

+------------+-------------+-----+
|       Place|     Hashtags|count|
+------------+-------------+-----+
|      Madrid|          FRA|   48|
|      Moscow|          FRA|   50|
|      Berlin|          FRA|   26|
|      Zagreb|          CRO|  104|
|      London|          ENG|  972|
|       Tokyo|          FRA|   50|
|       Paris|          FRA|  400|
|Buenos Aires|FIFAStadiumDJ|   88|
|       Accra|          FRA|  244|
|       Seoul|FIFAStadiumDJ|  100|
+------------+-------------+-----+



Определить столицы, в которых преобладающие хештеги из десяти наиболее упоминаемых

In [ ]:
finDF = sparkDF.select(F.split(sparkDF.Hashtags, ",").alias('Hashtags'),"Place")
finDF = finDF.select(explode(finDF.Hashtags).alias('Hashtags'),"Place")

finDF = finDF.select("Hashtags",F.split(sparkDF.Place, ",").alias('Place'))
finDF = finDF.select("Hashtags",explode(finDF.Place).alias('Place'))
finDF.show(10)


+-------------+--------+
|     Hashtags|   Place|
+-------------+--------+
|     WorldCup|   Accra|
|          POR|   Accra|
|          ENG|   Accra|
|     WorldCup|   Bogor|
|     worldcup|     NaN|
|   PowerByEXO| Zapopan|
|   PowerByEXO| Jalisco|
|     WorldCup| Zapopan|
|     WorldCup| Jalisco|
|FIFAStadiumDJ| Zapopan|
+-------------+--------+
only showing top 10 rows



In [ ]:
finDF = finDF.select("Place","Hashtags").collect()

rdd = sparkContext.parallelize(finDF)

rdd1 = rdd.take(5)
rdd1

[Row(Place='Accra', Hashtags='WorldCup'),
 Row(Place='Accra', Hashtags='POR'),
 Row(Place='Accra', Hashtags='ENG'),
 Row(Place='Bogor', Hashtags='WorldCup'),
 Row(Place='NaN', Hashtags='worldcup')]

In [ ]:
capitals = ['London','Moscow','Berlin','Madrid','Copenhagen','Lisbon','Bogota','Paris',\
            'Dakar','Tokyo','Stockholm','Buenos Aires','Tunis','Warsaw','Seoul','Seoul',\
            'Belgrade','Zagreb','Brussels','Accra']

filteredRDD = rdd.filter(lambda x: x[0] in capitals)

rdd1 = filteredRDD.take(5)
for f in rdd1:
    print("Place:"+ str(f[0]) +", Hashtags:"+f[1])

Place:Accra, Hashtags:WorldCup
Place:Accra, Hashtags:POR
Place:Accra, Hashtags:ENG
Place:Paris, Hashtags:WORLDCUP
Place:London, Hashtags:WorldCup


In [ ]:
wc = ['worldcup','Worldcup','WorldCup','WORLDCUP']

filteredRDD = filteredRDD.filter(lambda x: x[1] not in wc)
rdd1 = filteredRDD.take(5)
for f in rdd1:
    print("Place:"+ str(f[0]) +", Hashtags:"+f[1])

Place:Accra, Hashtags:POR
Place:Accra, Hashtags:ENG
Place:Buenos Aires, Hashtags:Cristiano
Place:Buenos Aires, Hashtags:Ronaldo
Place:Buenos Aires, Hashtags:Messi


In [ ]:
fdf = filteredRDD.toDF()
fdf = fdf.filter(~((isnan(fdf.Hashtags) | isnan(fdf.Place))))

fdf.show(5)

+------------+---------+
|       Place| Hashtags|
+------------+---------+
|       Accra|      POR|
|       Accra|      ENG|
|Buenos Aires|Cristiano|
|Buenos Aires|  Ronaldo|
|Buenos Aires|    Messi|
+------------+---------+
only showing top 5 rows



In [ ]:
fdfCount = fdf.groupBy('Place','Hashtags').count().orderBy("count", ascending=False)
fdfCount.show(10)

+------+-------------+-----+
| Place|     Hashtags|count|
+------+-------------+-----+
|London|          ENG|  972|
|London|          FRA|  824|
|London|   ThreeLions|  486|
|London|       ENGCRO|  429|
| Paris|          FRA|  400|
|London|          CRO|  394|
|London|WorldCupFinal|  333|
|London|       FRABEL|  330|
|London|       CRODEN|  265|
| Accra|          FRA|  244|
+------+-------------+-----+
only showing top 10 rows



In [ ]:
from pyspark.sql.window import Window
from pyspark.sql import Row
from pyspark.sql.functions import *

window = Window.partitionBy("Place").orderBy('tiebreak')

dfCount = fdfCount.withColumn('tiebreak', monotonically_increasing_id())\
 .withColumn('rank', rank().over(window))\
 .filter(col('rank') == 1).drop('rank','tiebreak')

dfCount.show()

+------------+-------------+-----+
|       Place|     Hashtags|count|
+------------+-------------+-----+
|      Madrid|          FRA|   48|
|      Lisbon|          FRA|    9|
|   Stockholm|          FRA|   24|
|    Belgrade|          FRA|   13|
|      Moscow|          FRA|   50|
|      Berlin|          FRA|   26|
|      Zagreb|          CRO|  104|
|      London|          ENG|  972|
|       Tokyo|          FRA|   50|
|       Tunis|          CRO|    2|
|       Paris|          FRA|  400|
|      Warsaw|          CRO|    6|
|    Brussels|       FRABEL|   53|
|Buenos Aires|FIFAStadiumDJ|   88|
|       Dakar|          SWE|    3|
|  Copenhagen|       CRODEN|   16|
|      Bogota|          CRO|   10|
|       Accra|          FRA|  244|
|       Seoul|FIFAStadiumDJ|  100|
+------------+-------------+-----+



In [ ]:
filteredHashtags.groupBy('Hashtags').count().orderBy("count", ascending=False).show(10)

+-------------+-----+
|     Hashtags|count|
+-------------+-----+
|          FRA|66128|
|          CRO|32274|
|WorldCupFinal|28446|
|          ENG|25165|
|       FRAARG|21413|
|       FRABEL|18862|
|       FRACRO|17982|
|          BEL|15728|
|          ARG|15241|
|FIFAStadiumDJ|14200|
+-------------+-----+
only showing top 10 rows



In [ ]:
import pyspark.sql.functions as F
listh = ["FRA","CRO","WorldCupFinal","ENG","FRAARG","FRABEL","FRACRO","BEL","ARG","FIFAStadiumDJ"]
dfCount.where(F.col("Hashtags").isin(listh)).show()

+------------+-------------+-----+
|       Place|     Hashtags|count|
+------------+-------------+-----+
|      Madrid|          FRA|   48|
|      Lisbon|          FRA|    9|
|   Stockholm|          FRA|   24|
|    Belgrade|          FRA|   13|
|      Moscow|          FRA|   50|
|      Berlin|          FRA|   26|
|      Zagreb|          CRO|  104|
|      London|          ENG|  972|
|       Tokyo|          FRA|   50|
|       Tunis|          CRO|    2|
|       Paris|          FRA|  400|
|      Warsaw|          CRO|    6|
|    Brussels|       FRABEL|   53|
|Buenos Aires|FIFAStadiumDJ|   88|
|      Bogota|          CRO|   10|
|       Accra|          FRA|  244|
|       Seoul|FIFAStadiumDJ|  100|
+------------+-------------+-----+

